# Data Science Capstone Project

# The Battle of the Neighborhoods - Week 1
                                                            - Ning Chu August, 2020

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-0">Introduction</a></span></li><li><span><a href="#Business-Problem" data-toc-modified-id="Business-Problem-1">Business Problem</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-2">Data</a></span></li><li><span><a href="#Problem-Statement" data-toc-modified-id="Problem-Statement-3">Problem Statement</a></span></li><li><span><a href="#Methodology" data-toc-modified-id="Methodology-4">Methodology</a></span></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-5">Analysis</a></span><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-5.1">Import libraries</a></span></li><li><span><a href="#Web-Scraping-Neighbourhood-Data" data-toc-modified-id="Web-Scraping-Neighbourhood-Data-5.2">Web Scraping Neighbourhood Data</a></span></li><li><span><a href="#Loading-and-Cleaning-Neighbourhood" data-toc-modified-id="Loading-and-Cleaning-Neighbourhood-5.3">Loading and Cleaning Neighbourhood</a></span></li><li><span><a href="#Obtaining-Neighbourhood-Coordinates" data-toc-modified-id="Obtaining-Neighbourhood-Coordinates-5.4">Obtaining Neighbourhood Coordinates</a></span></li><li><span><a href="#Define-Foursquare-Credentials-and-Version-use-API" data-toc-modified-id="Define-Foursquare-Credentials-and-Version-use-API-5.5">Define Foursquare Credentials and Version-use API</a></span></li><li><span><a href="#Exploring-the-first-neighborhood-in-our-dataframe." data-toc-modified-id="Exploring-the-first-neighborhood-in-our-dataframe.-5.6">Exploring the first neighborhood in our dataframe.</a></span></li><li><span><a href="#Exploring-all-Neighbourhoods" data-toc-modified-id="Exploring-all-Neighbourhoods-5.7">Exploring all Neighbourhoods</a></span></li><li><span><a href="#Analyzing-each-Neighbourhood" data-toc-modified-id="Analyzing-each-Neighbourhood-5.8">Analyzing each Neighbourhood</a></span></li></ul></li></ul></div>

## Introduction

Los Angeles often known by its initials L.A., is the largest city in California. With nearly four million people, it is the second-most populous city in the United State. Los Angeles is known for its Mediterranean climate, ethnic diversity, Hollywood entertainment industry, and its sprawling metropolis.

Many Taiwanese immigrants settled in California, leading to a number of bubble tea shops opening around Los Angeles.Bubble tea, popularly known as boba tea, is a Taiwanese drink.It is prepared with tea or milk as the base component. This tea is glazed with boba, which are soft, chewy, and gummy tapioca pearls extracted from cassava root.Since the ’90s, Los Angeles has a strong passion for milk tea studded with chewy pearls of boba. From Downtown to West Hollywood, Los Angeles has a large variety of boba houses offering both customizable teas and classic favorites.According to a new report published by Allied Market Research, the bubble tea market size was valued at 2.4 billion in 2019 and is estimated to reach 4.3 billion by 2027. **Therefore, the analysis and results of this project would interest stakeholders who are interested in opening a bubble tea shop in Los Angeles.**

## Business Problem

The objective of this Capstone project is to analyze and select the best locations in the Los Angeles to open a new Bubble Tea shop. Using Data Science methodology and instruments such as Data Analysis and Visualization, this project aims to provide solutions to answer the business question: Where in LA, should the investor open a Bubble Tea shop? 

## Data

**To solve the problem, we will need the following data:**
* Los Angeles data containing the neighborhoods and boroughs.
* Venue data, particularly data related to restaurants. This data will be used to perform further analysis of the neighborhoods
* Latitude and longitude coordinates of those neighborhoods.This is  required to plot the map and get the venue data.
* Average neighborhood rent

**Data Source**

* List of all neighborhoods in LA — https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_of_Los_Angeles
* Coordinates of all neighborhoods and venues — GeoPy Nominatim geocoding
* Number of restaurants and their type and location in every neighborhood — Foursquare API
* LA rent data — https://www.rentcafe.com/average-rent-market-trends/us/ca/los-angeles/

## Problem Statement
* What is / are the best location(s) for opening a Bubble Tea shop in LA?
* In what Neighborhood and/or borough should the investor open a Bubble Tea Shop to have the best chance of being successful?
* Where would I go in LA to have the best Bubble Tea?

## Methodology

1.  Collect data on the neighbourhoods of Los Angeles from the internet. There are no relevant datasets available for this and therefore, so we will need to be scraped from a webpage and processed into a dataframe. The location coordinates of each neighbourhood will used in  GeoPy Nominatim geolocator and appended to the neighbourhood data. 

2. Foursquare be used to  explore each of neighbourhoods and their venues using Foursquare location data. The venues of the neighbourhoods will be analyzed in detail and patterns will be discovered. This discovery of patterns will be carried out by grouping the neighbourhoods using k-means clustering. 

3. Each cluster will be examined and a decision will be made regarding which cluster fits the shareholder's requirements. 

4. Finally, if there are multiple neighbourhoods that fit these conditions, Los Angeles rent data can be used to influence the shareholder's decision.The results of the analysis will highlight potential neighbourhoods where a Bubble Tea Shop may be opened based on geographical location and proximity to competitors. 

5. Data of Bubble Tea Shop  will be sorted based on rankings in order to find where people go in can have the best Bubble Tea Shop.

## Analysis

### Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


###  Web Scraping Neighbourhood Data 

In [2]:
from bs4 import BeautifulSoup

url = requests.get('https://en.wikipedia.org/wiki/List_of_districts_and_neighbourhoods_of_Los_Angeles').text
soup = BeautifulSoup(url,"html.parser")

In [3]:
import re
lis = []
for li in soup.findAll('li'):
    if li.find(href="/wiki/Portal:Los_Angeles"):
        break
    if li.find(href=re.compile("^/wiki/")):
        lis.append(li)
    if li.text=='Pico Robertson[34]': #Pico Robertson is the only item on the list that does not have a hyperlink reference
        lis.append(li)

### Loading and Cleaning Neighbourhood

In [4]:
neigh = []
for i in range(0,len(lis)):
    neigh.append(lis[i].text.strip())
    
df = pd.DataFrame(neigh)
df.columns = ['Neighbourhood']

In [5]:
df.shape

(198, 1)

In [6]:
df

,Neighbourhood
0,Angelino Heights[1]
1,Angeles Mesa
2,Angelus Vista
3,Arleta[2][1]
4,Arlington Heights[2]
5,Arts District[3]
6,Atwater Village[2]
7,Baldwin Hills[1]
8,Baldwin Hills/Crenshaw[2]
9,Baldwin Village[1]


In [7]:
df['Neighbourhood'] = df.Neighbourhood.str.partition('[')[0] #Removes the citation and reference brackets
df['Neighbourhood'] = df.Neighbourhood.str.partition(',')[0] #Removes the alternatives for 'Bel Air'
df=df[df.Neighbourhood!='Baldwin Hills/Crenshaw'] #Removes redundancy as 'Baldwin Hills' and 'Crenshaw' exist already
df=df[df.Neighbourhood!='Hollywood Hills West'] #Removes redundancy as it has the same coordinates as 'Hollywood Hills'
df=df[df.Neighbourhood!='Brentwood Circle'] #Removes redundancy as it has the same coordinates as 'Brentwood'
df=df[df.Neighbourhood!='Wilshire Park'] #Removes redundancy as it has the same coordinates as 'Wilshire Center'
df.reset_index(inplace=True,drop=True)

### Obtaining Neighbourhood Coordinates 

The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe.

In [8]:
# define the data frame columns
column_names = ['Neighbourhood', 'Latitude', 'Longitude'] 

# instantiate the data frame
nhoods = pd.DataFrame(columns=column_names)

Use GeoPy Nominatim geolocator to get the latitude and longitude values of LA.

In [9]:
geolocator = Nominatim(user_agent="la_explorer",timeout=5)
for i in range(0,len(df)):
    
    address = df.Neighbourhood[i]+', Los Angeles'
    location = geolocator.geocode(address)
    if location == None:
        latitude = 0
        longitude = 0
    else:
        latitude = location.latitude
        longitude = location.longitude

    nhoods = nhoods.append({'Neighbourhood': df.Neighbourhood[i],
                                              'Latitude': latitude,
                                              'Longitude': longitude}, ignore_index=True)

In [10]:
nhoods

,Neighbourhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,Angeles Mesa,33.991402,-118.319520
2,Angelus Vista,0.000000,0.000000
3,Arleta,34.241327,-118.432205
4,Arlington Heights,34.043494,-118.321374
5,Arts District,34.041239,-118.234450
6,Atwater Village,34.118698,-118.262392
7,Baldwin Hills,34.010989,-118.337071
8,Baldwin Village,34.019456,-118.345910
9,Baldwin Vista,0.000000,0.000000


cleaning missing data

In [11]:
nhoods['Latitude']=nhoods['Latitude'].astype(float)
nhoods['Longitude']=nhoods['Longitude'].astype(float)

nhoods=nhoods[(nhoods.Latitude>33.5) & (nhoods.Latitude<34.4) & (nhoods.Longitude<-118)] 
nhoods.reset_index(inplace=True,drop=True)

In [12]:
nhoods

,Neighbourhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,Angeles Mesa,33.991402,-118.319520
2,Arleta,34.241327,-118.432205
3,Arlington Heights,34.043494,-118.321374
4,Arts District,34.041239,-118.234450
5,Atwater Village,34.118698,-118.262392
6,Baldwin Hills,34.010989,-118.337071
7,Baldwin Village,34.019456,-118.345910
8,Beachwood Canyon,34.122292,-118.321384
9,Bel Air,34.098883,-118.459881


Obtaining the coordinates of the center of LA: 

In [13]:
address = 'Los Angeles, USA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinates of Los Angeles, USA are 34.0536909, -118.2427666.


Create a map of LA with neighborhoods superimposed on top.

In [14]:
# create map of LA using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighbourhood in zip(nhoods['Latitude'], nhoods['Longitude'], nhoods['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3220cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_la)  
    
map_la

### Define Foursquare Credentials and Version-use API

In [15]:
CLIENT_ID = 'R01FNIIOXDYITMHBMFI3YZXPZ2IADYL5JEGZNHT33NSY5UFB' # your Foursquare ID
CLIENT_SECRET = 'JZ3QRJ05CESNNI3IEWFNI5AWDOUNL4OYY2GRWCAQCAPGS143' # your Foursquare Secret
VERSION = '20180601' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + "hidden" )
print('CLIENT_SECRET:' + "hidden")

Your credentails:
CLIENT_ID: hidden
CLIENT_SECRET:hidden


###  Exploring the first neighborhood in our dataframe.

In [16]:
neighbourhood_latitude = nhoods.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = nhoods.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = nhoods.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Angelino Heights are 34.0702889, -118.2547965.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f3cba307f58eb2f2f69848e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'East LA',
  'headerFullLocation': 'East LA, Los Angeles',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 26,
  'suggestedBounds': {'ne': {'lat': 34.074788904500004,
    'lng': -118.24937415757756},
   'sw': {'lat': 34.0657888955, 'lng': -118.26021884242243}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f75a626e5e8f16c87566797',
       'name': 'Halliwell Manor',
       'location': {'address': '1329 Carroll Ave',
        'lat': 34.069328534140894,
        'lng': -118.25416524263122,
        'labeledLatLngs': [{

Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Nearby venues of the first neighbourhood: -

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/ningchu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Halliwell Manor,Performing Arts Venue,34.069329,-118.254165
1,Eightfold Coffee,Coffee Shop,34.071245,-118.250698
2,Guisados,Taco Place,34.070262,-118.250437
3,"Michael Jackson's ""Thriller"" House (and Tree)",Historic Site,34.069557,-118.254599
4,Ototo,Sake Bar,34.072659,-118.251740


### Exploring all Neighbourhoods 

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
la_venues = getNearbyVenues(names=nhoods['Neighbourhood'],
                                   latitudes=nhoods['Latitude'],
                                   longitudes=nhoods['Longitude']
                                  )

In [23]:
print(la_venues.shape)
la_venues.head()

(3931, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Angelino Heights,34.070289,-118.254796,Halliwell Manor,34.069329,-118.254165,Performing Arts Venue
1,Angelino Heights,34.070289,-118.254796,Eightfold Coffee,34.071245,-118.250698,Coffee Shop
2,Angelino Heights,34.070289,-118.254796,Guisados,34.070262,-118.250437,Taco Place
3,Angelino Heights,34.070289,-118.254796,"Michael Jackson's ""Thriller"" House (and Tree)",34.069557,-118.254599,Historic Site
4,Angelino Heights,34.070289,-118.254796,Ototo,34.072659,-118.251740,Sake Bar


And how many venues were returned by Foursquare?

In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

26 venues were returned by Foursquare.


In [28]:
la_venues_count=la_venues.groupby('Neighbourhood').count()
la_venues_count.drop(la_venues_count.columns[[0,1,3,4,5]], axis=1,inplace=True)

In [29]:
la_venues_count.reset_index(inplace=True)

It makes sense to set up a tea shop in one of the more popular neighbourhoods so that the restaurant attracts the attention of a lot more people.
Therefore, a list of all the popular neighbourhoods i.e. the neighbourhoods with 10 or more venues is obtained

In [30]:
pop_neigh=la_venues_count[(la_venues_count.Venue>=10)]
pop_neigh.reset_index(drop=True,inplace=True)
pop_neigh

,Neighbourhood,Venue
0,Angelino Heights,26
1,Arlington Heights,12
2,Arts District,32
3,Atwater Village,41
4,Baldwin Hills,40
5,Beverly Grove,69
6,Beverly Hills Post Office,69
7,Beverly Park,16
8,Boyle Heights,31
9,Brentwood,23


Updating the venues data frame to include only the venues which are in popular neighbourhoods:

In [32]:
pop_list=pop_neigh['Neighbourhood'].values.tolist()

for i in range(0,len(la_venues)):

    if la_venues.iloc[i,0] not in pop_list:
        la_venues.iloc[i,0]='TO DROP'

la_venues=la_venues[la_venues.Neighbourhood!='TO DROP']
la_venues.reset_index(drop=True,inplace=True)


In [33]:
la_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Angelino Heights,34.070289,-118.254796,Halliwell Manor,34.069329,-118.254165,Performing Arts Venue
1,Angelino Heights,34.070289,-118.254796,Eightfold Coffee,34.071245,-118.250698,Coffee Shop
2,Angelino Heights,34.070289,-118.254796,Guisados,34.070262,-118.250437,Taco Place
3,Angelino Heights,34.070289,-118.254796,"Michael Jackson's ""Thriller"" House (and Tree)",34.069557,-118.254599,Historic Site
4,Angelino Heights,34.070289,-118.254796,Ototo,34.072659,-118.251740,Sake Bar
5,Angelino Heights,34.070289,-118.254796,Subliminal Projects,34.072290,-118.250737,Art Gallery
6,Angelino Heights,34.070289,-118.254796,Tsubaki,34.072938,-118.251298,Japanese Restaurant
7,Angelino Heights,34.070289,-118.254796,The Park's Finest BBQ,34.066519,-118.254291,BBQ Joint
8,Angelino Heights,34.070289,-118.254796,K Bakery,34.066124,-118.254059,Bakery
9,Angelino Heights,34.070289,-118.254796,Bar Henry,34.069062,-118.250465,Cocktail Bar


In [34]:
print('There are {} uniques categories.'.format(len(la_venues['Venue Category'].unique())))

There are 319 uniques categories.


### Analyzing each Neighbourhood 

In [36]:
# one hot encoding
la_onehot = pd.get_dummies(la_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to data frame
la_onehot['Neighbourhood'] = la_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

,Neighbourhood,ATM,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Football Field,College Residence Hall,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sta